# Merk Coding Challenge Section 1

Below is my submission for the Merk coding challenge. If you have any questions, please email me at mosecodes@gmail.com or refer to my resume for further contact information. 

_Note: I edited my pyplate.py file on line 2713 to use <code>.map</code> instead of <code>.applymap</code> for the DataFrame visualizations, since applymap has been deprecated, and the warning messages clogged the output._

## INSTRUCTIONS

To fully meet the requirements of the coding challenge, the below recipe must be performed twice. Once for the 60 degree run and once for the 80 degree run.

The provided output shows the final mmol counts for a sample of the starting materials, all ligands, and the palladium catalyst. It also shows the total reaction volume in _uL_. Solvents are dispersed across the rows, with rows A and E having solvent 1, B and F having solvent 2, and so on. 

### PART A

In [1]:
# PART A
from pyplate import Substance, Container, Plate, Recipe
import numpy as np
import random

random.seed(10)
        

# CREATE SUBSTANCES FOR RXS-----------------------------------------------------------------------------
# solvents
toluene = Substance.liquid(name='toluene', density=0.8623, mol_weight=92.141)
glyme = Substance.liquid(name='glyme', density=0.8683, mol_weight=90.122)
MTBE = Substance.liquid(name='MTBE', density=0.7404, mol_weight=88.150)
dichloroethane = Substance.liquid(name='dichloroethane', density=1.253, mol_weight=98.95)
solvents = np.array([toluene, glyme, MTBE, dichloroethane])

# ligands
xphos = Substance.solid(name='XPhos', mol_weight=476.72)
sphos = Substance.solid(name='SPhos', mol_weight=410.53)
dppf = Substance.solid(name='dppf', mol_weight=554.391)
ligands = np.array([xphos, sphos, dppf])

# catalyst
catalyst = Substance.solid(name='Palladium(II) acetate', mol_weight=224.51)

# random starting materials
a = []
b = []
for i in range(1,13):
    a_weight = random.randrange(80, 150)
    b_weight = random.randrange(80, 150)
    a_sub = Substance.solid(f'A{i}', mol_weight=a_weight)
    b_sub = Substance.solid(f'B{i}', mol_weight=b_weight)
    a.append(a_sub)
    b.append(b_sub)

# arrays for containers of starting solutions
a_arr = np.array(a)
b_arr = np.array(b)

   
# PREPARE RECIPE-----------------------------------------------------------------------------------------
# create initail plates
source_plate_1 = Plate('source_plate_1', max_volume_per_well='500 uL')
source_plate_2 = Plate('source_plate_2', max_volume_per_well='500 uL')
dest_plate_1 = Plate('dest_plate_1', max_volume_per_well='500 uL')
dest_plate_2 = Plate('dest_plate_2', max_volume_per_well='500 uL')

# initialize recipe with plate
# fill plates with solvent and ligand
recipe = Recipe()
recipe.uses([source_plate_1, source_plate_2, dest_plate_1, dest_plate_2])


# PREPARE A AND B SOLUTIONS------------------------------------------------------------------------------
# prepare 1.1 Bi equivalent of Ai
recipe.start_stage(name='PREP STARTING MATERIAL IN SOURCE PLATES')

for i in range(12):
    for row, solvent in enumerate(solvents):
        # fill both source plates with starting materials
        # each column gets the same starting materials for all rows
        # each row gets a different solvent, depending on the row number
        
        a_solution = recipe.create_solution(solute=a_arr[i], solvent=solvent, name=f'A{i+1} in {solvent}',
                                                concentration='1 M', total_quantity='1 mL')
        b_solution = recipe.create_solution(solute=b_arr[i], solvent=solvent, name=f'B{i+1} in {solvent}',
                                                concentration='1.1 M', total_quantity='1 mL')
        # transfer solutions to source plates
        # at selected concentrations 100 uL will give 0.1mmol and 0.11mmol, respectively
        well_1 = (row+1, i+1)
        well_2 = (row+5, i+1)
        
        # fill source plate 1
        recipe.transfer(source=a_solution, destination=source_plate_1[[well_1, well_2]], quantity='100 uL')
        recipe.transfer(source=b_solution, destination=source_plate_1[[well_1, well_2]], quantity='100 uL')
        # fill source plate 2
        recipe.transfer(source=a_solution, destination=source_plate_2[[well_1, well_2]], quantity='100 uL')
        recipe.transfer(source=b_solution, destination=source_plate_2[[well_1, well_2]], quantity='100 uL')

recipe.end_stage(name='PREP STARTING MATERIAL IN SOURCE PLATES')
recipe.start_stage(name='TRANSFER STARTING MATERIALS TO DESTINATION PLATES')

# send plates to dest_plate
recipe.transfer(source_plate_1, dest_plate_1, '100 uL')
recipe.transfer(source_plate_2, dest_plate_2, '100 uL')

recipe.end_stage(name='TRANSFER STARTING MATERIALS TO DESTINATION PLATES')

# ADD LIGAND/CATALYST/SOLVENT TO DESTINATION PLATES------------------------------------------------------
recipe.start_stage(name='PREP CATALYST IN SOURCE PLATES')

# create catalyst solutions
# add 0.01 mmol of catalyst to source plates
cat_solutions = []
for row, solvent in enumerate(solvents):
    cat_solution = recipe.create_solution(solute=catalyst, solvent=solvent, name=f'{catalyst.name} in {solvent}',
                                          concentration='1 M', total_quantity='1 mL')
    cat_solutions.append(cat_solution)
    recipe.transfer(source=cat_solution, destination=source_plate_1[row+1], quantity='10 uL')
    recipe.transfer(source=cat_solution, destination=source_plate_1[row+5], quantity='10 uL')
    recipe.transfer(source=cat_solution, destination=source_plate_2[row+1], quantity='10 uL')
cat_solutions_arr = np.array(cat_solutions)

recipe.end_stage(name='PREP CATALYST IN SOURCE PLATES')
recipe.start_stage(name='PREP LIGAND IN SOURCE PLATES')

# add 0.015 mmol  ligand to source plates
for row_group, ligand in enumerate(ligands):
       for row, solvent in enumerate(solvents):
        # three ligands and four solvents
        # every row gets the same amount (mmol) of each 
        lig_solution = recipe.create_solution(solute=ligand, solvent=solvent, name=f'{ligand.name} in {solvent}',
                                              concentration='1.5 M', total_quantity='1 mL')
        row_num = row_group + row + 1
        
        # add solutions to destination plates
        if row_group == 0:    
            # at the concentrations selected, 10uL will give 0.015mmol of ligand and 0.01mmol of catalyst
            recipe.transfer(source=lig_solution, destination=source_plate_1[row_num], quantity='10 uL')
            recipe.fill_to(solvent=solvent, destination=source_plate_1[row_num], quantity='100 uL')
        elif row_group == 1:
            recipe.transfer(source=lig_solution, destination=source_plate_1[row_num+3], quantity='10 uL')
            recipe.fill_to(solvent=solvent, destination=source_plate_1[row_num+3], quantity='100 uL')
        else:
            recipe.transfer(source=lig_solution, destination=source_plate_2[row_num-row_group], quantity='10 uL')
            recipe.fill_to(solvent=solvent, destination=source_plate_2[row_num-row_group], quantity='100 uL')

recipe.end_stage(name='PREP LIGAND IN SOURCE PLATES')
recipe.start_stage(name='TRANSFER CATALYST/LIGAND TO DESTINATION PLATES')

# send plates to dest_plate
recipe.transfer(source_plate_1, dest_plate_1, '100 uL')
recipe.transfer(source_plate_2, dest_plate_2, '100 uL')

recipe.end_stage(name='TRANSFER CATALYST/LIGAND TO DESTINATION PLATES')


recipe.bake()

        

separator = '='
print('CATALYST MMOL IN EACH WELL\n')
cat_plate_1 = recipe.visualize(what=dest_plate_1, mode='final', substance=catalyst, unit='mmol')
cat_plate_1.caption = 'Catalyst in Well Plate 1 (mmol)'
cat_plate_2 = recipe.visualize(what=dest_plate_2, mode='final', substance=catalyst, unit='mmol')
cat_plate_2.caption = 'Catalyst in Well Plate 2 (mmol)'
display(cat_plate_1, cat_plate_2)

print(f'{separator*80}')
print('STARTING MATERIAL A MMOL IN FIRST COLUMN\n')
a_plate_1 = recipe.visualize(what=dest_plate_1, mode='final', substance=a_arr[0], unit='mmol')
a_plate_1.caption = 'A1 in Well Plate 1 (mmol)'
a_plate_2 = recipe.visualize(what=dest_plate_2, mode='final', substance=a_arr[0], unit='mmol')
a_plate_2.caption = 'A1 in Well Plate 2 (mmol)'
display(a_plate_1, a_plate_2)

print(f'{separator*80}')
print('STARTING MATERIAL B MMOL IN FIRST COLUMN\n')
b_plate_1 = recipe.visualize(what=dest_plate_1, mode='final', substance=b_arr[0], unit='mmol')
b_plate_1.caption = 'B1 in Well Plate 1 (mmol)'
b_plate_2 = recipe.visualize(what=dest_plate_2, mode='final', substance=b_arr[0], unit='mmol')
b_plate_2.caption = 'B1 in Well Plate 2 (mmol)'
display(b_plate_1, b_plate_2)

print(f'{separator*80}')
print('TOTAL VOLUME IN EACH WELL\n')
vol_display_1 = recipe.visualize(what=dest_plate_1, mode='final', unit='uL')
vol_display_1.caption = 'Total Volume for Plate 1 (uL)'
vol_display_2 = recipe.visualize(what=dest_plate_2, mode='final', unit='uL')
vol_display_2.caption = 'Total Volume for Plate 2 (uL)'
display(vol_display_1, vol_display_2)

print(f'{separator*80}')
print('LIGAND XPHOS MMOL IN EACH WELL OF PLATE 1\n')
xphos_plate_1 = recipe.visualize(what=dest_plate_1, mode='final', substance=xphos, unit='mmol')
xphos_plate_1.caption = 'XPhos in Well Plate 1 (mmol)'
display(xphos_plate_1)

print(f'{separator*80}')
print('LIGAND SPHOS MMOL IN EACH WELL OF PLATE 1\n')
sphos_plate_1 = recipe.visualize(what=dest_plate_1, mode='final', substance=sphos, unit='mmol')
sphos_plate_1.caption = 'SPhos in Well Plate 1 (mmol)'
display(sphos_plate_1)

print(f'{separator*80}')
print('LIGAND DPPF MMOL IN EACH WELL OF PLATE 2\n')
dppf_plate_2 = recipe.visualize(what=dest_plate_2, mode='final', substance=dppf, unit='mmol')
dppf_plate_2.caption = 'DPPF in Well Plate 2 (mmol)'
display(dppf_plate_2)

print(f'{separator*80}')
print('RECIPE STAGES\n')
recipe.stages.pop('all')
for stage in recipe.stages:
    print(f'{stage}\n-')

print(f'{separator*80}')
print('NUMBER OF STEPS\n')
num_steps = len(recipe.steps)
print(f'Total number of steps: {num_steps}\n')
if num_steps > 100:
    print("That's a lot of steps! Oh well, let's keep going\n")

print(f'{separator*80}')
print('END\n')


# you may uncomment the below code to get all the details of each recipe step 

# print(f'{separator*80}')
# print('RECIPE STEPS\n')
# for i, step in enumerate(recipe.steps):
#     print(f'Step: {i+1}')
#     display(step)
# TODO: go through each substance and ensure all measurements are correct for both plates

# END ---------------------------------------------------------------------------------------------------


CATALYST MMOL IN EACH WELL



,1,2,3,4,5,6,7,8,9,10,11,12
A,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
B,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
C,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
D,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
E,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
F,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
G,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
H,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010


,1,2,3,4,5,6,7,8,9,10,11,12
A,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
B,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
C,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
D,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
E,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
F,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
G,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
H,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


STARTING MATERIAL A MMOL IN FIRST COLUMN



,1,2,3,4,5,6,7,8,9,10,11,12
A,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
B,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
C,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
D,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
E,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
F,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
G,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
H,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


...

LIGAND SPHOS MMOL IN EACH WELL OF PLATE 1



,1,2,3,4,5,6,7,8,9,10,11,12
A,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
B,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
C,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
D,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
E,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015
F,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015
G,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015
H,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015


LIGAND DPPF MMOL IN EACH WELL OF PLATE 2



,1,2,3,4,5,6,7,8,9,10,11,12
A,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015
B,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015
C,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015
D,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015
E,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
F,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
G,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
H,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


RECIPE STAGES

PREP STARTING MATERIAL IN SOURCE PLATES
-
TRANSFER STARTING MATERIALS TO DESTINATION PLATES
-
PREP CATALYST IN SOURCE PLATES
-
PREP LIGAND IN SOURCE PLATES
-
TRANSFER CATALYST/LIGAND TO DESTINATION PLATES
-
NUMBER OF STEPS

Total number of steps: 344

That's a lot of steps! Oh well, let's keep going

END



### Part B

To incorporate a 'tags' feature, my approach would be 4-fold.

<ol>
    <li> Modify the Substance class:
    <ul>
        <li>Add new <code>tags</code> attribute to the Substance class to store the labels/tags associated with the substance.</li>
        <li>Modify the constructor of the Substance class to accept an optional <code>tags</code> parameter and initialize the <code>tags</code> attribute accordingly.</li>
        <li>Add a method to the Substance class to add or remove tags from the substance.
    </ul>
    </li>
    <li> Modify the Recipe class:
    <ul>
        <li>Add a method to the Recipe class to associate <code>tags</code> with substances.</li>
        <li>Modify the <code>.create_solution</code> method of the Recipe class to handle the relative quantities specified using <code>tags</code>. This method should calculate the actual quantity based on the <code>tags</code> and the specified factor.</li>
    </ul>
    </li>
    <li> Modify the Container and Plate classes:
    <ul>
        <li>Modify the <code>.transfer</code> method of the Container and Plate classes to handle the relative quantities specified using <code>tags</code>. This should effectively do the same thing as the modified method in the Recipe class.</li>
    </ul>
    </li>
    <li> Modify the visualization methods:
    <ul>
        <li>Modify the visualization methods to accept <code>tags</code> as parameters, or display tags when visualizing Substances.</li>
    </ul>
    </li>
</ol>

To handle physical limitations, the methods for calculating the relative quantities should check the <code>max_volume</code> attributes of the Containers they are being moved into to ensure that the new solution will be within the volume limits. The Unit class can be used to help with this, and ensure consistency in calculations. 

